In [16]:
import pandas as pd
import flair
from bpemb import BPEmb
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
from sklearn.decomposition import TruncatedSVD
import regex as re
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, BytePairEmbeddings
#import spacy

In [2]:
full_files_assignment = [f"data/assignment/{x}/assignment.txt" for x in os.listdir('data/assignment')]
full_files_data_assignment = [open(x).read() for x in full_files_assignment if not 'annot' in x and not '.DS' in x]
all_text = '\n'.join(full_files_data_assignment) # split into test/train

In [3]:
multi_space = re.compile("[_ \t]+")
non_ascii = re.compile("[^\x00-\x7FåäöÅÄÖ\s\n\p{Punct}]+")
punct = re.compile('([\p{Punct}\n])+')
#splitter = re.compile("[\n\.]")
#t = all_text.replace('/', ' ')
t = non_ascii.sub('', all_text)
t = punct.sub(' \g<1> ', t)
t = multi_space.sub(' ', t)
t = t.replace('\n', '.')
t = t.lower()

In [4]:
t[:300]

'ad system designer / system architect - automotive , referens 20922 . ansök här . uppdragsbeskrivning och krav . we are now looking for a system designer / system architect to our clients autonomous drive and active safety function department . you have documented experience of system design within '

In [5]:
keywords = {'wireshark', 'cyber', 'docker', 'kanban', 'latest technologies', 'teaching skills', 'jenkins', 'sae j2847', 'system design', 'android', 'ccc', 'measurements', 'thermal conditioning', 'product development process', 'sales', 'verification tests', 'plc', 'test automation', 'private equity', 'data handling', 'maven', 'in-vehicle software', 'project management', 'validate', 'autonomous driver', 'insurances', 'drug', 'problem solver', 'infotainment', 'sensor fusion', 'ats', 'documentation', 'integration transformation', 'test planning', 'wallet-sizing analysis', 'software requirement', 'propulsion', 'test plans', 'canalyzer', 'nvidia', 'research and development', 'team efficiency', 'modems', 'automated test', 'matlab simulink', 'spice', 'lead', 'canoe', 'stk', 'control', 'product owner', 'lin', 'signal processing', 'insurance', 'frontend', 'customer', 'mil', 'backend', 'github', 'database', 'engineering tests', 'web services', 'automotive standards', 'valuation gap', 'university degree', 'software', 'cabel harness', 'application maintenance', 'sw test', 'collaborative', 'engineering', 'pcb and manufacturing', 'documenting processes', 'planning', 'safety requirement', 'coordinate', 'microprocessor system', 'interfaces', 'javascript', 'communication skills', 'user intent algorithms', 'target setting', 'rtos', 'hil', 'devops', 'digital design', 'manufacturing', 'electrical engineering', 'fund', 'sensors', 'soc', 'front-end', '5+', 'kola', 'network', 'gerrit', 'financial', 'signals', 'recruitment', 'low-level', 'phd', '>5', 'ci/cd', 'mcu', 'design for immunity', 'bus chassis', 'integration testing', 'ad/adas', 'stakeholders', 'm.sc.', 'equity analyst', 'bts', 'gpu development', 'back-end', 'artifactory', 'verbal', 'sae j3068', 'biopharmaceutical', 'offsite', 'schematic design', 'coverage model', 'etas inca', 'standardized requirement work', 'can', 'networking', 'data-centric', 'stake holder communication', 'erp system', 'electronics', 'saber', 'logistics systems', 'telecom', 'mechatronics', 'adas', 'polyspace', 'controller', 'quality assurance', 'coaching', 'canalayzer', 'c', 'invoicing', 'machine learning', 'train management', 'system lead', 'bachelor', 'technical', 'manage', 'windows', 'cmmi', 'project manager', 'cross-functional', 'management consulting', 'warranty', 'powertrain', 'end-to-end', 'bash', 'iso 15118', 'drivetrain', 'test programs', 'systems development', 'coordinating', 'robot framework', 'hazard analysis', 'oas', 'system testing', 'sensor perception layer', 'functional safety manager', 'report', 'efficiency', 'html', 'emc certification', 'uwb', 'it', 'diagnostics', 'esd', 'designer', 'oracle', 'testweaver', 'developer', 'healthcare', 'portfolio strategy', 'technical debt', 'confluence', 'unit testing', 'architecture', 'client discussion', '7740', 'sts', 'supply chain designer', 'electric motors', 'global coordination', 'aerospace', 'academic degree', 'e2e', 'efficient', 'flexray', 'financial modelling', 'embedded software', 'certification tests', 'design', 'business experience', 'configuration management', 'php', 'mechanical engineering', 'elektra', 'cloud', 'computer engineering', 'retail', 'vehicle electronics', 'eeprom', 'drug substance', 'high speed signals', 'power company', 'industrialization', 'filter designs', 'test', 'sap', 'css', 'system control', 'implementation', 'inverters', '5-8 years', 'validation', 'ad', 'electronics designer', 'application development', 'activism defense', 'front end', 'back end', 'sil1', 'cable harness engineer', 'it solutions', 'apriso', 'r&d', 'sql', 'wso2', 'software architect', 'track record', 'cd', 'business development', 'security', 'phone as key', 'jira', 'banking', 'system simulators', 'embedded system', 'hedging', 'sil', 'embedded systems', "master's degree", 'r3', 'pl', 'autonomous', 'app-centric', 'team lead', 'autosar', 'ai', 'bidding', 'traction', 'safe', 'budget', 'databases', 'international', 'rest api', 'saas', 'swedish', 'order management', 'automation', 'cts', 'budgeting', 'cameras', 'ccs', 'git', 'embedded', 'analog design', 'sw', 'emc', 'quality', 'customer value', 'vehicle certification', 'e-mobility', 'team leader', 'system verification', 'product manager', 'batteries', 'driving', '> 5', 'international business', 'scm', 'financial institutions', 'advanced driver assistance system', 'carve-out', 'team work', 'b.sc.', 'battery', 'dc/dc', 'linux', 'functional development', 'research & development', 'iso26262', 'home office', 'vehicle', 'operational team management', 'workshops', 'warranties', 'optimization', 'debugging', 'carweaver', 'hardware', 'pcb-a', 'italian', 'system development', 'din 70121', 'claim management', 'software developer', 'review', 'business', 'logistics', 'asil', 'computer science', 'cyber security', 'verification', 'chargers', 'vts', 'initiative', 'keycloak', 'qnx', 'carcom', 'ultrasonic perception', 'strategic vulnerabilities', 'payments', 'design for emissions', 'c/c++', 'bid teams', 'epsm', 'simulink', 'hil testing', 'angular', 'python', 'restful', 'sensor signaling', 'financing', 'digital key', 'fintech', 'operational', 'user experience', 'leader', 'railway', 'pcb', 'polarion', 'portuguese', 'continuous integration', 'radars', 'design guidelines', 'ethernet', 'electronic designer', 'vehicle dynamics', 'mba', 'matlab', 'economy', 'testing methodology', 'capl', 'java', 'master of science', 'ram', '10+', 'automotive', 'traction batteries', 'management', 'driving license', 'inca', 'english', 'pm', 'ci', 'system architecture', 'architect', 'stk calculations', 'coordination', 'gtest', 'cybersecurity', 'sell-side', 's4hana', 'intellij', 'ul4600', 'cable harness designer', 'finance', 'agile', 'nand', 'communication', 'canalyser', 'aspice', 'sw architect', 'ad platform', 'technology', 'engineering physics', 'invoice', 'test methods', 'safety', 'construction', 'customer requirement', 'mälardalen', 'software requirements', 'robot', "bachelor's degree", 'nfc', 'ble', 'integrity', 'rest', 'risk assessment', 'big data', 'c++', 'supply strategy', 'project leader', 'change management', 'design alternatives', 'spring', 'scrum', 'system architect'}

In [6]:
tokens = t.split(' ')
tokens[:5]

['ad', 'system', 'designer', '/', 'system']

In [7]:
sorted([x.count(' ') for x in keywords], reverse=True)[:3]

[3, 2, 2]

In [10]:
tagged_token = []
prev_index = 0
for i in range(len(tokens)):
    if i < prev_index:
        continue
    token = tokens[i]
    num_tokens = 0
    
    if i < len(tokens) - 3 and ' '.join(tokens[i:i+4]) in keywords:
        num_tokens = 4
    elif i < len(tokens) - 2 and ' '.join(tokens[i:i+3]) in keywords:
        num_tokens = 3
    elif i < len(tokens) - 1 and ' '.join(tokens[i:i+2]) in keywords:
        num_tokens = 2
    elif token in keywords:
        num_tokens = 1
    
    if num_tokens > 0:
        for j in range(num_tokens):
            if j == 0:
                tagged_token.append(f"{tokens[i+j]} B-KEY")
            else:
                tagged_token.append(f"{tokens[i+j]} I-KEY")
        prev_index = i + num_tokens
    else:
        prev_index = i
        tagged_token.append(f"{token} 0")

In [11]:
tagged_token[:20]

['ad B-KEY',
 'system 0',
 'designer B-KEY',
 '/ 0',
 'system B-KEY',
 'architect I-KEY',
 '- 0',
 'automotive B-KEY',
 ', 0',
 'referens 0',
 '20922 0',
 '. 0',
 'ansök 0',
 'här 0',
 '. 0',
 'uppdragsbeskrivning 0',
 'och 0',
 'krav 0',
 '. 0',
 'we 0']

In [12]:
with open('data/keyword_ds/ds.txt', 'w') as f:
    f.write('\n'.join(tagged_token))

In [17]:
columns = {0: 'text', 1: 'ner'}
corpus: Corpus = ColumnCorpus('data/keyword_ds', columns, train_file='ds.txt')
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

2021-06-03 18:59:52,362 Reading data from data/keyword_ds
2021-06-03 18:59:52,363 Train: data/keyword_ds/ds.txt
2021-06-03 18:59:52,365 Dev: None
2021-06-03 18:59:52,365 Test: None
Dictionary with 7 tags: <unk>, O, B-KEY, 0, I-KEY, <START>, <STOP>


In [ ]:
embedding_types = [
	BytePairEmbeddings('sv'),
	FlairEmbeddings("sv-forward"),
	FlairEmbeddings("sv-backward")
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
										embeddings=embeddings,
										tag_dictionary=tag_dictionary,
										tag_type=tag_type,
										use_crf=True)